# Proyek Analisis Sentimen terhadap Ulasan Pengguna Aplikasi BRImo
- **Nama:** Chatarina Evangelista Sitorus
- **Email:** chatarinaevas304@gmail.com
- **ID Dicoding:** dicodingchatzz

Proyek ini bertujuan untuk memahami persepsi pengguna terhadap aplikasi BRImo melalui analisis sentimen berdasarkan ulasan di Google Play Store.

## **1. Import Packages/Libraries yang Digunakan**

In [7]:
# Install sebelum import
!pip install nltk
!pip install Sastrawi
!pip install wordcloud
!pip install tensorflow

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/375.7 MB ? eta -:--:--
   ---------------------------------------- 0.8/375.7 MB 1.5 MB/s eta 0:04:06
   ---------------------------------------- 1.0/375.7 MB 1.6 MB/s eta 0:03:59
   ---------------------------------------- 1.3/375.7 MB 1.3 MB/s eta 0:04:39
   ---------------------------------------- 1.8/375.7 MB 1.6 MB/s eta 0:03:54
   ---------------------------------------- 2.1/375.7 MB 1.6 MB/s eta 0:03:50
   ---------------------------------------- 2.6/375.7 MB 1.8 MB/s eta 0:03:26
   ---------------------------------------- 3.1/375.7 MB 1.8 MB/s eta 0:03:24
   ---------------------------------------- 3.7/375.7 MB 1.9 MB/s eta 0:03:18
   ---------------------------------------- 4.2/375.

In [8]:
# Import library umum 
import pandas as pd  
import numpy as np  
import datetime as dt  # Untuk manipulasi waktu dan tanggal
import re  # Ekspresi reguler (regex)
import string  # Operasi pada karakter, termasuk tanda baca
pd.options.mode.chained_assignment = None  # Menonaktifkan warning chained assignment
seed = 0
np.random.seed(seed)  # Set seed agar hasil reproducible

# Visualisasi 
import matplotlib.pyplot as plt  
import seaborn as sns

# Visualisasi berbasis teks 
from wordcloud import WordCloud  # Membuat word cloud dari teks

# Preprocessing teks dasar 
from nltk.stem import WordNetLemmatizer  # Lematisasi kata (khusus Bahasa Inggris)
from nltk.tokenize import word_tokenize  # Memecah kalimat menjadi kata
from nltk.corpus import stopwords  # Daftar kata umum yang sering dibuang
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Preprocessing & evaluasi model ML 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score

# Stemming dan stopword Bahasa Indonesia 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Untuk stemming Bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Stopword remover Indo

# Deep learning dengan TensorFlow/Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Flatten, Bidirectional, SimpleRNN, Dropout, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, SpatialDropout1D, LSTM
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


import requests
import csv
from io import StringIO


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chata\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chata\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chata\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


## **2. Memuat Dataset**

In [10]:
# Load dataset dari file CSV
df = pd.read_csv("Dataset/dataset_ulasan_brimo.csv")

# Menampilkan seluruh data
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,b942ba12-2658-4f03-9f5a-b4a8d6c379b2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"BRImo menawarkan kemudahan transaksi, tampilan...",5,927,2.79.0,2025-03-12 14:05:04,"Hai Sobat BRI, mohon maaf atas ketidaknyamanan...",2025-03-12 14:30:05,2.79.0
1,ee7e5f6f-2265-4d7d-967b-2419ab4edd03,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Buat yg keganggu dikit2 update tak kasi tau, u...",5,2002,2.79.0,2025-03-13 21:10:57,Terima kasih atas ulasannya. Semoga aplikasi B...,2025-03-14 08:10:42,2.79.0
2,92845e58-a03e-4c82-abfd-e8133548983f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Login via scan fingerprint, malah muncul error...",5,1351,2.79.0,2025-03-11 20:01:33,"Hai Sobat BRI, mohon maaf atas kendala yang di...",2025-03-16 15:32:05,2.79.0
3,75a3d673-8796-460f-89a6-bc423c17ffa7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi BRImo ini cukup membantu, tidak perlu...",5,51,2.80.0,2025-03-28 06:41:48,"Hai Sobat BRI, mohon maaf atas kendala yang di...",2022-11-23 17:50:27,2.80.0
4,f39fce9a-42b1-4980-866d-e14b5b63dbcb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"rawan security nya, menurut saya. Lapisan pass...",5,467,2.78.0,2025-02-18 04:07:09,"Hai, Sobat BRI. Mohon maaf atas kendala yang d...",2025-02-18 05:04:49,2.78.0
...,...,...,...,...,...,...,...,...,...,...,...
29995,d2e12047-9b93-40d7-bb72-0b82773139a6,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sangat bermanfaat dan membantu sekali,,, terim...",5,15,2.55.0,2024-01-14 16:31:49,Terima kasih sudah menggunakan aplikasi BRImo ...,2024-01-14 19:20:02,2.55.0
29996,0a6f855f-7975-465c-a0fa-8667b04eab10,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya suka aplikasi ini kalau lagi ada saldonya...,5,0,2.39.0,2023-04-14 11:19:03,Terima kasih atas ulasannya. Semoga aplikasi B...,2023-05-09 16:31:00,2.39.0
29997,b75d8c2d-b388-4499-9094-095716326b64,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Mantap ad promo tersembunyi,5,0,2.71.1,2024-10-30 19:50:55,"Hai, Sobat BRI. Terima kasih sudah menggunakan...",2025-01-26 16:19:29,2.71.1
29998,6494295d-412d-44cf-8fcf-bebf784eed8b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sangat puas dan sangat mudah untuk transaksi.,5,0,NaN,2025-01-30 13:24:34,"Hai, Sobat BRI. Terima kasih sudah menggunakan...",2025-01-30 14:18:40,NaN


## **3. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. 


In [11]:
df.sample(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
8225,5c33bcc8-26e4-4232-bc70-95a03cbf152f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Smp saat ini blm ada kendala saat menggunakan ...,5,4,2.68.0,2024-09-21 11:28:52,Terima kasih sudah menggunakan aplikasi BRImo ...,2024-09-22 16:33:48,2.68.0
10794,32df5923-ae16-4d5b-b8df-a425b7fb772a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Cukup memuaskan...,5,0,2.80.0,2025-03-18 08:19:47,"Hai Sobat BRI, terima kasih sudah menggunakan ...",2025-03-18 09:15:12,2.80.0
9163,3fe44c46-f6a4-4727-9809-db165d5c03d0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Take video berkali kali gagal terus..... Alasa...,5,0,2.25.0,2022-08-09 20:29:10,"Hai, Sobat BRI. Mohon maaf atas ketidaknyamana...",2025-01-08 18:06:26,2.25.0
26591,9d52d703-bd9f-4ae0-990e-cbe0bcfdd46e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,suka banget sama aplikasinya BRImo sangat bagu...,5,0,2.1.0,2021-03-02 11:53:26,Terima kasih atas ulasannya. Semoga aplikasi B...,2022-01-23 10:38:06,2.1.0
6631,1f8704a2-c2ac-4163-9206-5aed7f351b05,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,memudahkan untuk bertransaksi,5,0,2.80.0,2025-03-23 22:02:57,"Hai Sobat BRI, terima kasih atas ulasannya. Se...",2025-03-24 06:37:02,2.80.0
